In [1]:
from scipy.fftpack import dct,idct
import scipy
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
from decimal import Decimal
import operator
from math import cos,sin,pi

In [2]:
#==============ماتریس چندی سازی
Z = [[16, 11, 10, 16, 24, 40, 51, 61],
     [12, 12, 14, 19, 26, 58, 60, 55],
     [14, 13, 16, 24, 40, 57, 69, 56],
     [14, 17, 22, 29, 51, 87, 80, 62],
     [18, 22, 37, 56, 68 ,109 ,103 ,77],
     [24, 35, 55, 64, 81 ,104 ,113 ,92],
     [49, 64, 78, 87, 103, 121, 120, 101],
     [72, 92, 95, 98, 112, 100, 103, 99]]
temporal = {} # موقتی یا ac
frecuencia = {} # فرکانس  dc

g = 0;

def dct2(a):
    new_matriz = np.zeros((8,8));
    b = np.zeros((8,8));
    #############################################################################################
    #
    #   از هر یک از عناصر 128 را کم می کنیم
    #
    #############################################################################################
    for i in range(8):
        for j in range(8):
            b[i][j] = a[i][j] - 128;
    ##############################################################################################
    #
    #   dct به بلوک 8x8 اعمال می شود
    #
    ##############################################################################################
    c = scipy.fftpack.dct( scipy.fftpack.dct( b, axis=0, norm='ortho' ), axis=1, norm='ortho' )


    ##############################################################################################
    #
    #   بلوک 8x8 عادی است
    #
    ##############################################################################################

    for i in range(8):
        for j in range(8):
            new_matriz[i][j] = np.fix(c[i][j]/Z[i][j]) # تقسیم بر z
    
    return new_matriz


In [3]:
#===============> بارگذاری تصویر
matrix = Image.open("00_lena.jpg")
#===============> نمایش تصویر اصلی
matrix.show()


In [4]:
#===============>  تبدیل تصویر به خاکستری
matrix = matrix.convert('L')
matrix.save("01_gray.jpg")
matrix.show()

In [5]:
#===============> ارتفاع و عرض تصویر را بدست می آوریم
alto, ancho = matrix.size

#================>تبدیل ماتریس به آراییه
a = np.asarray(matrix,dtype=np.float32)

#================>(انتقال سطح) کم کردن تصویر بدست آمده از  128
alternativo  = a;
alternativo = alternativo - 128;

#================>  نمایش تصویر در حالت انتقال سطح پیدا کرده
Image.fromarray(alternativo.astype(np.uint8)).save("02_subtracted_128.jpg")
I = Image.open("02_subtracted_128.jpg");
I.show()
#================>  ساخت یک ماتریس صفر 256* 256 برای پردازش نتیجه
im2 = np.zeros((256,256))

In [6]:
##########################################################################################
#
#                   """اجرای الکوریتم فشرده سازی jpeg"""
#
##########################################################################################

In [7]:
#==================> جدا کردن 8*8  از تصویر و انجام تبدیل فوریه کوسینوسی روی آن
for i in range(0,alto,8):
    for j in range(0,ancho,8):
        im2[i:(i+8),j:(j+8)] = dct2(a[i:(i+8),j:(j+8)])


In [8]:
#==================> دخیره تصویر بدست آمده از dct دو بعدی
Image.fromarray(im2.astype(np.uint8)).save("03_dct.jpg")
I = Image.open("03_dct.jpg");
I.show()

In [9]:
#===================> انجام مراحل زیکزاکی برای هر بلوک
def zigzag(matrix):
    global g
    con = 0
    matrix = np.array(matrix)
    rows=8
    columns=8
    aux = np.zeros((1,64)) #ساخت یک آرایه یک بعدی برای ذخیره المان ها بصورت زیکزاکی
    x = np.zeros((1,64))
    y = np.zeros((1,64))

    solution=[[] for i in range(rows+columns-1)]
    
    for i in range(rows): 
        for j in range(columns): 
            sum=i+j
            
            if(sum%2 ==0):
                solution[sum].insert(0,matrix[i][j]) 
            
            else:
                solution[sum].append(matrix[i][j])
              
    solucion = solution.reverse()
#    f = open("zig.txt", "a")
    for i in solution:
        for j in i:
            if j == -0.0:
                j = abs(j)
                aux[0,con] = j
            else:
                aux[0,con] = j
            con = con + 1;
    
    indice = 0;

    for i in range(0,64):
        if(aux[0,i]!= 0):
            indice = i
            break

    for i in range(indice,64):
       
        temporal[g] = aux[0,i];

        if aux[0,i] in frecuencia:
            pass
        
        else:
            frecuencia[aux[0,i]] = aux[0,i]
        g = g+1

for i in range(0,alto,8):
        for j in range(0,ancho,8):
            zigzag(im2[i:(i+8),j:(j+8)])


#==================>ترکیب بلوک های 8*8 زیکزاکی و دخیره آن در فایل متنی
f = open("03_dct.txt","w")
for i in range(0,alto):
    for j in range(0,ancho):
        if im2[i,j] == -0.0:
            f.write(str(abs(im2[i,j]))+" ")
        else:
            f.write(str(im2[i,j])+" ")
    f.write("\n")

In [10]:

#==================>ترکیب بلوک های 8*8 زیکزاکی و دخیره آن در فایل متنی
f = open("03_dct.txt","w")
for i in range(0,alto):
    for j in range(0,ancho):
        if im2[i,j] == -0.0:
            f.write(str(abs(im2[i,j]))+" ")
        else:
            f.write(str(im2[i,j])+" ")
    f.write("\n")

In [11]:
# لیست فرکانس ها
keys = list(frecuencia.keys())
elementos = list(temporal.values())
tam = len(keys);

probabilidades = {}
#=====================> احتمال رخ دادن هر عدد(آنتروپی)
for i in keys:

    telemento = float(elementos.count(i));
    probabilidades[i] = telemento/float(len(elementos));
#==================>ذخیره سازی احتمالات در یک فایل متنی
    pos = 0;
    file = open("04_probability.txt","w")
    for i in probabilidades:
        file.write(str(keys[pos]) +"\t"+ str(probabilidades.get(i,i))+"\n");
        pos+= 1
print("فشرده سازی با موفقیت انجام شد و فایل احتممالات  در فایل نتایج ذحیره سازی گردید")


فشرده سازی با موفقیت انجام شد و فایل احتممالات  در فایل نتایج ذحیره سازی گردید


In [12]:
##########################################################################################
#
#                   """ اجرای کد هافمن"""
#
##########################################################################################

In [13]:
result_probabilidades = "04_probability.txt"
fichero = "04_probability.txt"

def compresor_huffman():
	probabilidades = {}	
	tabla_codigos = {}
	
	line = ""
	archivo = open(result_probabilidades,'r');
	for i in  archivo:
		line = i.split("\t")

		if(line[0] == "space"):
			pass
		elif(line[0]=="salto"):
			pass
		else:
			probabilidades[line[0]] = float(line[1].rstrip())
	archivo.close()
	tabla_codigos = huffmanCode(probabilidades)
	save_codes(tabla_codigos)
	
	with open(fichero,'r') as txt, open("05_compressed.dat",'wb') as salida:
		txt = (txt.read().rstrip()).lower()
		
		encoded_text = TextEncode(tabla_codigos,txt)
		padded_encodded = PadEncode(encoded_text);
		CadenaBits = GeneraBitArray(padded_encodded)
		salida.write(bytes(CadenaBits))


#================>تابعی که امکان ترتیب احتمالات را فراهم می کند
def ordenar_probabilidades(dic):
#====================>تابع مرتب شده دیکشنری احتمالات را از پایین به بالاترین مرتب می کند 
	ordenado = sorted(dic.items(), key = operator.itemgetter(1), reverse=False)
	
	
	# با توجه به دو احتمال اول یک تاپل را با کلیدهای دیکشنری برگرداند
    # که در این حالت دو نماد کم احتمالی هستند.
	
	return ordenado[0][0], ordenado[1][0]

def huffmanCode(dic):

   # حالت پایه فقط دو نماد ، 0 یا 1 را خودسرانه اختصاص دهید
	if(len(dic) == 2):
		return dict(zip(dic.keys(), ['0', '1']))

    # با ادغام کمترین احتمال ، توزیع جدیدی ایجاد می کنیم
	p_copy = dic.copy()
	
	K1, K2 = ordenar_probabilidades(dic)

	p1, p2 = p_copy.pop(K1), p_copy.pop(K2)
	p_copy[K1 + K2] = p1 + p2
	c = huffmanCode(p_copy)
	ca1a2 = c.pop(K1 + K2)
	c[K1], c[K2] = ca1a2 + '0', ca1a2 + '1'
	return c
#################################################################################
# کد را در یک فایل متنی ساده ذخیره کنید
def save_codes(dic):
	file = open("06_coded.txt","w")
	for i in dic:
		if i == '\n':
			file.write("salto"+"\t"+ dic.get(i,i)+"\n");
		else:
			file.write(dic.get(0,i) +"\t"+ dic.get(i,i)+"\n");
	file.close();
###############################################################################

def TextEncode(codes, texto):

	encode_text = ""
	#print texto.split()
	for ch in texto.split():
		if ch in codes:
			#print(ch, codes[ch])
			encode_text += codes[ch]

	return encode_text

def PadEncode(encoded):
	padding = 8 - len(encoded) % 8

	for i in range(padding):
		encoded += "0"

	padded_info = "{0:08b}".format(padding)
	encoded = padded_info + encoded

	return encoded

def GeneraBitArray(cadena_binaria):
	if(len(cadena_binaria) % 8 != 0):
		exit(0)

	Cbits = bytearray()

	for i in range(0, len(cadena_binaria), 8):
		byte = cadena_binaria[i:i+8]
		Cbits.append(int(byte, 2))
	
	return Cbits


compresor_huffman()
print("\n\n")

sizefile = os.path.getsize("04_probability.txt")
sizefile2 = os.path.getsize("05_compressed.dat")
sizefile= sizefile/(1024*1024.0)
sizefile2 = sizefile2/(1024*1024.0)
porcentaje  = ((sizefile2/sizefile) * 100)


print ("text original: " +str("04_probability.txt")+ " size: "+ str(sizefile)+" MB")
print ("File compressed: compressed.dat size: " + str(sizefile2)+" MB")
print("Archivo "+"04_probability.txt"+" compressed en "+ str(round(porcentaje))+"%")
print("compressed:) !!\n\n")





text original: 04_probability.txt size: 0.0022230148315429688 MB
File compressed: compressed.dat size: 9.5367431640625e-05 MB
Archivo 04_probability.txt compressed en 4%
compressed:) !!




In [14]:
fichero = "06_coded.txt"
def descomprimir():
#=================> پرونده ای را که می خواهیم از حالت فشرده خارج کنیم بارگذاری کنید ، برای نشان دادن پرونده از 'rb' استفاده کنید
#=================> که به آن مراجعه می شود در حالت باینری خوانده می شود
	file = open(fichero,'rb')
	bit_string = ""

#================> خواندن هر خط از فایل و انجام مراحل رمزگشایی
	byte = file.read(1)
	contador = 1;
	while(byte != ''):
		try:
			byte = ord(byte)
			contador = contador + 1;
			bits = bin(byte)[2:].rjust(8,'0')
			bit_string += bits
			byte = file.read(1)
		except:
			break

	TextEncode = quitar_bit_padding(bit_string) 
	TextDescomprimido = decodificarTexto(TextEncode)
	write_file(TextDescomprimido)

#نتیجه رمز گشایی را در یک فایل متنی قرار می دهیم.
def write_file(TextDescomprimido):

#فایل متنی را برای نوشتن درون ان باز می کنیم(اگر وجود نداشته باشد ایجاد می شود)
	file = open("07_matrix.txt","w")
	file.write(TextDescomprimido)

#سپس فایل را می بندیم
	file.close()


# صفرهای اضافه شده را حذف کنید ، هنگامی که یک رشته کمی کامل نشد ،
# این در پرونده ایجاد شده_کد_هفمن است.
def quitar_bit_padding(encode_text):

	padding = encode_text[:8]
	paddingExtra = int(padding,2)

	encode_text = encode_text[8:]

	text = encode_text[:-1*paddingExtra]

	return text
"""
این روش متن رمزگذاری شده با کدهای هافمن را برای آن رمزگشایی می کند
از دفترچه راهنما استفاده می کند که در این حالت به صورت داخلی بارگذاری می شود.
"""
def decodificarTexto(TextEncode):
# یک فرهنگ لغت خالی ایجاد کنید
	tablecode = {}
	invertcode = {}

# دفترچه کد را باز کرده و در فرهنگ لغت ذخیره کنید.
	codes = open("06_coded.txt","r")

	for i in  codes:
# فاصله را از رشته کاراکتر خوانده شده حذف می شود.
		line = i.split("\t")

		if(line[0] == " "):
			tablecode[' '] = line[1].rstrip()
		elif(line[0]=="salto"):
			tablecode['\n'] = line[1].rstrip()
		else:
			tablecode[line[0]] = line[1].rstrip()
# ترتیب فرهنگ لغت را معکوس می کنیم تا بتوانیم با کدها جستجو کنیم
	
	kkey = list(tablecode.keys());
	vvalues = list(tablecode.values());
	ntablecode = {}
	for i in range(len(vvalues)):
		ntablecode[vvalues[i]] = kkey[i]
	
	tablecode = ntablecode;

	codes.close() 
	current_code = ""
	decodetext = ""
# متن رمزگذاری شده را تکرار کنید
	con = 0;
	for bit in TextEncode:
		current_code += bit
		# بررسی کنید که آیا کد خوانده شده در کتاب کد موجود است
		if (current_code in tablecode):
			con += 1;
		# کاراکتر مربوط به کد خوانده شده را به کاراکتر اختصاص دهید.
			char = tablecode[current_code]
		# هرکدام از کاراکترهایی را که پیدا می شوند ، می خواند
			if(con == 256):
				decodetext += char+"\n"
				con = 0;
			else:	
				decodetext += char+" "
			current_code = ""
	# رشته را با متن باز شده بازگرداند.
	return decodetext

descomprimir()

print("\n\nفایل کد ها با نام "+str(fichero)+"ایجاد شد!!\n")



فایل کد ها با نام 06_coded.txtایجاد شد!!



In [15]:
""" این عملکرد مراحل بازیابی تصویر فشرده شده توسط JPEG را انجام می دهد"""
def idct2(matrix):
	b = np.zeros((8,8));
	new_matrix = np.zeros((8,8));

	"""بلوک های 8x8 با ماتریس نرمال سازی ضرب می شوند"""
	for i in range(8):
		for j in range(8):
			b[i][j] = matrix[i][j] * Z[i][j];

	"""محاسبه IDCT انجام می شود"""
	c = scipy.fftpack.idct(scipy.fftpack.idct(b,axis=0, norm='ortho'),axis=1, norm='ortho')

	"""128 در اینجا عنصر بلوک 8x8 اضافه شده است"""
	for i in range(8):
		for j in range(8):
			new_matrix[i][j] = c[i][j] + 128;
	return new_matrix

"""ما ماتریس بدست آمده از فرآیند JPEG را بارگذاری می کنیم"""
matrix = np.loadtxt("03_dct.txt",dtype=float)
matrix = np.array(matrix)
imagen = np.zeros((256,256))

for i in range(0,256,8):
	for j in range(0,256,8):
		imagen[i:(i+8), j:(j+8)] = idct2(matrix[i:(i+8), j:(j+8)])

"""تصویر بازیابی شده نمایش داده می شود"""
Image.fromarray(imagen.astype(np.uint8)).save("08_compressed.jpg")
I = Image.open("08_compressed.jpg");
I.show()
